In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### Prepare Data

In [1]:
from google.colab import files
upload = files.upload()

Saving disaster_train.csv to disaster_train.csv


In [2]:
from google.colab import files
upload = files.upload()

Saving disaster_val.csv to disaster_val.csv


In [3]:
from google.colab import files
upload = files.upload()

Saving disaster_test.csv to disaster_test.csv


In [4]:
from google.colab import files
upload = files.upload()

Saving train.csv to train.csv


In [5]:
from google.colab import files
upload = files.upload()

Saving test.csv to test.csv


In [6]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

In [7]:
!pip install transformers

     |████████████████████████████████| 1.4MB 8.1MB/s 
     |████████████████████████████████| 2.9MB 10.8MB/s 
     |████████████████████████████████| 890kB 12.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=c277584ec9b5c75a6e4a838ecd691e3dd16d3f87dc50e723861dd4eb58f7d662
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


The transformer has already been trained with a specific vocabulary, which means we need to train with the exact same vocabulary and also tokenize our data in the same way that the transformer did when it was initially trained.

Luckily, the transformers library has tokenizers for each of the transformer models provided. In this case we are using the BERT model which ignores casing (i.e. will lower case every word). We get this by loading the pre-trained bert-base-uncased tokenizer.

In [8]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

The tokenizer has a vocab attribute which contains the actual vocabulary we will be using. We can check how many tokens are in it by checking its length.

In [9]:
len(tokenizer.vocab)

30522

Another thing we need to handle is that the model was trained on sequences with a defined maximum length - it does not know how to handle sequences longer than it has been trained on. We can get the maximum length of these input sizes by checking the max_model_input_sizes for the version of the transformer we want to use. In this case, it is 512 tokens.

In [10]:
max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']

In [11]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [12]:
# Now I write a function of replacing words matching regular expression
import re
replacement_patterns = [
 (r'won\'t', 'will not'),
 (r'can\'t', 'cannot'),
 (r'i\'m', 'i am'),
 (r'ain\'t', 'is not'),
 (r'(\w+)\'ll', '\g<1> will'),
 (r'(\w+)n\'t', '\g<1> not'),
 (r'(\w+)\'ve', '\g<1> have'),
 (r'(\w+)\'s', '\g<1> is'),
 (r'(\w+)\'re', '\g<1> are'),
 (r'(\w+)\'d', '\g<1> would')
]
class Regularexpreplacer:
  def __init__(self):
    self.pattern = [(re.compile(match),repl) for (match,repl) in replacement_patterns]
  def replace(self,text):
    k = text
    for (p,repl) in self.pattern:
      k = re.sub(p,repl,k)
    return k

Now , we clear the unusual charecters , stop word('and','or','the',etc) , websides name as most as possible

In [13]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
# at the case of tokenization we tokenize the word and washout english stop word at the same time
def tokenize_word(sentence):
    pure_sentence = re.sub(r'[-,?!@:#;+0-9/$\*]','',sentence)#replace these symbol by space
    pure_sentence1 = re.sub(r'[\'.]','',pure_sentence)
    pure_sentence2 = re.sub(r'[][]','',pure_sentence1)
    pure_sentence3 = re.sub(r'\x89..','',pure_sentence2)
    pure_sentence4 = re.sub(r'[<=>|]','',pure_sentence3)
    pure_sentence5 = re.sub(r'[()]','',pure_sentence4)
    token_word = word_tokenize(Regularexpreplacer().replace(pure_sentence5.lower()))
    word = list(filter(lambda x:not x.startswith('https' and 'http'),token_word))# replace thw websides that start with https or http
    english_stopword = set(stopwords.words('english')) # set of english stopword
    filter_word = [w for w in word if w not in english_stopword] # washout english stop word such as 'and','or','the',etc
    filter_words = filter_word[:max_input_length-2]
    return filter_words

We can numericalize tokens using our vocabulary using tokenizer.convert_tokens_to_ids.

In [14]:
t = tokenize_word('Hello WORLD how ARE yoU?')
indexes = tokenizer.convert_tokens_to_ids(t)
print(indexes)

[7592, 2088]


The transformer was also trained with special tokens to mark the beginning and end of the sentence, detailed here. As well as a standard padding and unknown token. We can also get these from the tokenizer.

Note: the tokenizer does have a beginning of sequence and end of sequence attributes (bos_token and eos_token) but these are not set and should not be used for this transformer.

In [15]:
init_token = tokenizer.cls_token
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, eos_token, pad_token, unk_token)

[CLS] [SEP] [PAD] [UNK]


We can get the indexes of the special tokens by converting them using the vocabulary...

In [16]:
init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

101 102 0 100


Torchtext takes in raw data in the form of text files, csv/tsv files, json files, and directories (as of now) and converts them to Datasets. Datasets are simply preprocessed blocks of data read into memory with various fields. They are a canonical form of processed data that other data structures can use.

Torchtext then passes the Dataset to an Iterator. Iterators handle numericalizing, batching, packaging, and moving the data to the GPU. Basically, it does all the heavy lifting necessary to pass the data to a neural network.




---


Let’s compile a list of tasks that text preprocessing must be able to handle. All 'Y' marked lines are functionalities provided by Torchtext.

1. Train/Val/Test Split: seperate your data into a fixed train/val/test set (not used for k-fold validation)

2. File Loading: load in the corpus from various formats('Y')

3. Tokenization: break sentences into list of words('Y')

4. Vocab: generate a vocabulary list('Y')

5. Numericalize/Indexify: Map words into integer numbers for the entire corpus('Y')

6. Word Vector: either initialize vocabulary randomly or load in from a pretrained embedding, this embedding must be “trimmed”, meaning we only store words in our vocabulary into memory.('Y')

7. Batching: generate batches of training sample (padding is normally happening here)('Y')

8. Embedding Lookup: map each sentence (which contains word indices) to fixed dimension word vectors

#### Declaring the Fields
Torchtext takes a declarative approach to loading its data: you tell torchtext how you want the data to look like, and torchtext handles it for you.

The way you do this is by declaring a Field. The Field specifies how you want a certain (you guessed it) field to be processed


>>field is  a A regular field that defines preprocessing and postprocessing

In [135]:
from torchtext import data
TEXT = data.Field(batch_first = True, 
                  use_vocab = False, 
                  tokenize = tokenize_word, 
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx,
                  )
LABEL = data.LabelField(use_vocab = False,dtype = torch.float)

#### Constructing the Dataset

The fields know what to do when given raw data. Now, we need to tell the fields what data it should work on. This is where we use Datasets.

There are various built-in Datasets in torchtext that handle common data formats. For csv/tsv files, the TabularDataset class is convenient.




---
For the TabularDataset, we pass in a list of (name, field) pairs as the fields argument. The fields we pass in must be in the same order as the columns. For the columns we don't use, we pass in a tuple where the field element is None.





In [136]:
fields = [(None,None),(None,None),(None,None),("text",TEXT) ,("target",LABEL)]

In [137]:
from torchtext import datasets
from torchtext import data
train_data,val_data,test_data = data.TabularDataset.splits(path ='/content',
                                        train = 'disaster_train.csv',
                                        validation = 'disaster_val.csv',
                                        test = 'disaster_test.csv',
                                        format = 'csv',
                                        fields = fields,
                                        skip_header = True)

it's instructive to take a look inside our Dataset. Datasets can be indexed and iterated over like normal lists, so let's see what the first element looks like:

In [138]:
train_data[1].__dict__.keys()

dict_keys(['text', 'target'])

In [139]:
train_data[1].text # by this process we can access 'text' attriute of index 1 of training example

[3224, 2543, 2379, 2474, 100, 100, 2710]

In [140]:
train_data[2].text # by this process we can access 'text' attribute of index 2 of training example

[3901, 2356, 7713, 2173, 19488, 3738, 13982, 7713, 2173, 4449, 3517]

In [141]:
train_data[2].target

'1'

Then, we can create the iterators after defining our batch size and device.

By default, the train data is shuffled each epoch, but the validation/test data is sorted. However, TorchText doesn't know what to use to sort our data and it would throw an error if we don't tell it.

There are two ways to handle this, you can either tell the iterator not to sort the validation/test data by passing sort = False, or you can tell it how to sort the data by passing a sort_key. A sort key is a function that returns a key on which to sort the data on. For example, lambda x: x.s will sort the examples by their s attribute(example of attribute represent here 'n' , 'c', 'v', 'text', 'target' of train_data,val_data,test_data), i.e their quote. Ideally, you want to use a sort key as the BucketIterator will then be able to sort your examples and then minimize the amount of padding within each batch.

We can then iterate over our iterator to get batches of data. Note how by default TorchText has the batch dimension second.

In [142]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 128
train_iterator,valid_iterator,test_iterator = data.BucketIterator.splits((train_data,val_data,test_data),sort = False,batch_size = BATCH_SIZE,device = device)

In [143]:
print('Train:')
for batch in train_iterator:
    print(batch)
    
print('Valid:')
for batch in valid_iterator:
    print(batch)
    
print('Test:')
for batch in test_iterator:
    print(batch)

Train:

[torchtext.data.batch.Batch of size 128]
	[.text]:[torch.cuda.LongTensor of size 128x20 (GPU 0)]
	[.target]:[torch.cuda.FloatTensor of size 128 (GPU 0)]

[torchtext.data.batch.Batch of size 128]
	[.text]:[torch.cuda.LongTensor of size 128x26 (GPU 0)]
	[.target]:[torch.cuda.FloatTensor of size 128 (GPU 0)]

[torchtext.data.batch.Batch of size 128]
	[.text]:[torch.cuda.LongTensor of size 128x19 (GPU 0)]
	[.target]:[torch.cuda.FloatTensor of size 128 (GPU 0)]

[torchtext.data.batch.Batch of size 128]
	[.text]:[torch.cuda.LongTensor of size 128x23 (GPU 0)]
	[.target]:[torch.cuda.FloatTensor of size 128 (GPU 0)]

[torchtext.data.batch.Batch of size 128]
	[.text]:[torch.cuda.LongTensor of size 128x21 (GPU 0)]
	[.target]:[torch.cuda.FloatTensor of size 128 (GPU 0)]

[torchtext.data.batch.Batch of size 128]
	[.text]:[torch.cuda.LongTensor of size 128x19 (GPU 0)]
	[.target]:[torch.cuda.FloatTensor of size 128 (GPU 0)]

[torchtext.data.batch.Batch of size 128]
	[.text]:[torch.cuda.LongTe

In [144]:
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


### Bulid Model

Next, we'll load the pre-trained model, making sure to load the same model as we did for the tokenizer.

In [145]:
from transformers import BertTokenizer,BertModel
bert = BertModel.from_pretrained('bert-base-uncased')

In [146]:
import torch.nn as nn

class BERTGRUSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        
        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
                
        with torch.no_grad():
            embedded = self.bert(text)[0]
                
        #embedded = [batch size, sent len, emb dim]
        
        hidden1, hidden = self.rnn(embedded)
        
        #hidden = [n layers * n directions, batch size, emb dim]
        
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
                
        #hidden = [batch size, hid dim]
        
        output = self.out(hidden)
        
        #output = [batch size, out dim]
        
        return output

In [125]:
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25
model = BERTGRUSentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)

### Train the Model

In [147]:
import torch.optim as optim
lr = 0.001
optimizer = optim.Adam(model.parameters(),lr = lr)

In [148]:
criterion = nn.BCEWithLogitsLoss()

In [149]:
model = model.to(device)
criterion = criterion.to(device)

In [150]:
def binary_accuracy(pred,y):
  rounded_pred = torch.round(torch.sigmoid(pred))
  correct = (rounded_pred == y).float()
  acc = correct.sum()/len(correct)
  return acc

In [151]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    clip = 4
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.target)
        
        acc = binary_accuracy(predictions, batch.target)
        
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [152]:
def evaluate(model, iterator, criterion,l):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            l.append(torch.round(torch.sigmoid(predictions)))
            loss = criterion(predictions, batch.target)
            
            acc = binary_accuracy(predictions, batch.target)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [153]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [154]:

N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    l=[]
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion,l)
        
    end_time = time.time()
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut6-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 6s
	Train Loss: 0.429 | Train Acc: 80.81%
	 Val. Loss: 0.377 |  Val. Acc: 84.28%
Epoch: 02 | Epoch Time: 0m 6s
	Train Loss: 0.404 | Train Acc: 82.07%
	 Val. Loss: 0.416 |  Val. Acc: 80.34%
Epoch: 03 | Epoch Time: 0m 6s
	Train Loss: 0.409 | Train Acc: 82.42%
	 Val. Loss: 0.409 |  Val. Acc: 82.18%
Epoch: 04 | Epoch Time: 0m 6s
	Train Loss: 0.378 | Train Acc: 83.98%
	 Val. Loss: 0.418 |  Val. Acc: 82.46%
Epoch: 05 | Epoch Time: 0m 6s
	Train Loss: 0.365 | Train Acc: 84.35%
	 Val. Loss: 0.395 |  Val. Acc: 82.96%


In [155]:

model.load_state_dict(torch.load('tut6-model.pt'))
l=[]
test_loss, test_acc = evaluate(model, test_iterator, criterion,l)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.365 | Test Acc: 83.82%


In [156]:
l

[tensor([0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1.,
         1., 0., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1.,
         1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 0., 1., 0., 0., 1., 1.,
         1., 0.], device='cuda:0'),
 tensor([0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0.,
         0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.,
         1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.,
         0., 0., 1., 1., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 

In [157]:
fields1 = [(None,None),(None,None),(None,None),("text",TEXT) ,("target",LABEL)]
fields2 = [(None,None),(None,None),(None,None),("text",TEXT) ,(None,None)]

In [158]:
from torchtext import datasets
from torchtext import data
train_data_pred, = data.TabularDataset.splits(path ='/content',
                                      train = 'train.csv',
                                      format = 'csv',
                                      fields = fields1,
                                      skip_header = True)
test_data_pred, = data.TabularDataset.splits(path ='/content',
                                      test = 'test.csv',
                                      format = 'csv',
                                      fields = fields2,
                                      skip_header = True)

In [159]:
train_data_pred[1].__dict__.keys()

dict_keys(['text', 'target'])

In [160]:
test_data_pred[1].__dict__.keys()

dict_keys(['text'])

In [161]:
train_data[1].text

[3224, 2543, 2379, 2474, 100, 100, 2710]

In [162]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 128
train_iterator,test_iterator  = data.BucketIterator.splits((train_data_pred,test_data_pred),sort = False,batch_size = BATCH_SIZE,device = device)

In [163]:
print('Train:')
for batch in train_iterator:
    print(batch)

print('Test:')
for batch in test_iterator:
    print(batch)

Train:

[torchtext.data.batch.Batch of size 128]
	[.text]:[torch.cuda.LongTensor of size 128x21 (GPU 0)]
	[.target]:[torch.cuda.FloatTensor of size 128 (GPU 0)]

[torchtext.data.batch.Batch of size 128]
	[.text]:[torch.cuda.LongTensor of size 128x21 (GPU 0)]
	[.target]:[torch.cuda.FloatTensor of size 128 (GPU 0)]

[torchtext.data.batch.Batch of size 128]
	[.text]:[torch.cuda.LongTensor of size 128x25 (GPU 0)]
	[.target]:[torch.cuda.FloatTensor of size 128 (GPU 0)]

[torchtext.data.batch.Batch of size 128]
	[.text]:[torch.cuda.LongTensor of size 128x19 (GPU 0)]
	[.target]:[torch.cuda.FloatTensor of size 128 (GPU 0)]

[torchtext.data.batch.Batch of size 128]
	[.text]:[torch.cuda.LongTensor of size 128x19 (GPU 0)]
	[.target]:[torch.cuda.FloatTensor of size 128 (GPU 0)]

[torchtext.data.batch.Batch of size 128]
	[.text]:[torch.cuda.LongTensor of size 128x21 (GPU 0)]
	[.target]:[torch.cuda.FloatTensor of size 128 (GPU 0)]

[torchtext.data.batch.Batch of size 128]
	[.text]:[torch.cuda.LongTe

In [164]:
def evaluate_pred(model, iterator, criterion,l1):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            l1.append(torch.round(torch.sigmoid(predictions)))


In [166]:
N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    end_time = time.time()
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
    torch.save(model.state_dict(), 'tut6-model1.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 6s
	Train Loss: 0.333 | Train Acc: 86.12%
Epoch: 02 | Epoch Time: 0m 6s
	Train Loss: 0.303 | Train Acc: 87.43%
Epoch: 03 | Epoch Time: 0m 6s
	Train Loss: 0.289 | Train Acc: 88.17%
Epoch: 04 | Epoch Time: 0m 6s
	Train Loss: 0.268 | Train Acc: 89.05%
Epoch: 05 | Epoch Time: 0m 6s
	Train Loss: 0.248 | Train Acc: 90.06%
Epoch: 06 | Epoch Time: 0m 6s
	Train Loss: 0.229 | Train Acc: 90.79%
Epoch: 07 | Epoch Time: 0m 6s
	Train Loss: 0.217 | Train Acc: 90.98%
Epoch: 08 | Epoch Time: 0m 6s
	Train Loss: 0.208 | Train Acc: 91.80%
Epoch: 09 | Epoch Time: 0m 6s
	Train Loss: 0.184 | Train Acc: 92.71%
Epoch: 10 | Epoch Time: 0m 6s
	Train Loss: 0.175 | Train Acc: 92.85%


In [167]:
model.load_state_dict(torch.load('tut6-model1.pt'))
p=[]
evaluate_pred(model, test_iterator, criterion,p)

In [168]:
p

[tensor([0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1.,
         0., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0.,
         1., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 1.,
         0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0.,
         0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0.,
         0., 0.], device='cuda:0'),
 tensor([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0.,
         0., 0., 1., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 0., 0., 0., 1., 1., 1., 1.,
         1., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1., 

In [169]:
len(p)

26

In [170]:
y_pred=[]
for i in range(len(p)):
  y_pred.extend(np.array(p[i].detach().to(torch.device("cpu"))))

In [171]:
y_pred

[0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0

In [172]:
len(y_pred)

3263

In [173]:
y_hat = pd.DataFrame(np.array(y_pred))

In [100]:
y_hat

,0
0,1.0
1,1.0
2,1.0
3,0.0
4,1.0
...,...
3258,1.0
3259,1.0
3260,1.0
3261,0.0


In [102]:
df = pd.read_csv("test.csv")

In [103]:
df

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [106]:
df_test = df.iloc[:,3:4]

In [107]:
df_test

,text
0,Just happened a terrible car crash
1,"Heard about #earthquake is different cities, s..."
2,"there is a forest fire at spot pond, geese are..."
3,Apocalypse lighting. #Spokane #wildfires
4,Typhoon Soudelor kills 28 in China and Taiwan
...,...
3258,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,Storm in RI worse than last hurricane. My city...
3260,Green Line derailment in Chicago http://t.co/U...
3261,MEG issues Hazardous Weather Outlook (HWO) htt...


In [108]:
kframe = [df_test,y_hat]

In [110]:
ki=pd.concat(kframe,axis=1)

In [111]:
ki=pd.DataFrame(ki)

In [112]:
ki

,text,0
0,Just happened a terrible car crash,1.0
1,"Heard about #earthquake is different cities, s...",1.0
2,"there is a forest fire at spot pond, geese are...",1.0
3,Apocalypse lighting. #Spokane #wildfires,0.0
4,Typhoon Soudelor kills 28 in China and Taiwan,1.0
...,...,...
3258,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...,1.0
3259,Storm in RI worse than last hurricane. My city...,1.0
3260,Green Line derailment in Chicago http://t.co/U...,1.0
3261,MEG issues Hazardous Weather Outlook (HWO) htt...,0.0


In [113]:
ki.to_csv('disaster_prediction.csv')